In [1]:
import pandas as pd
import numpy as np
import gcsfs
import recordlinkage as rl
import matching as m
import helpers as h
%load_ext autoreload
%autoreload 2

In [2]:
from gcsfs.core import GCSFileSystem
gcs = GCSFileSystem('courseradataeng', token='/home/jupyter/.config/gcloud/application_default_credentials.json') 

In [3]:
with gcs.open('soleadify_sample_data/unified_matched_data/matches_full.csv') as f:
    matches_full = pd.read_csv(f, quotechar='"', escapechar='\\')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (1,3,6,9,12,24,25,27,30,33,36) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
matches_full.columns

Index(['address__facebook', 'address__google', 'category__facebook',
       'category__google', 'category__website', 'city_clean__facebook',
       'city_clean__google', 'city_clean__website', 'country_clean__facebook',
       'country_clean__google', 'country_clean__website',
       'description__facebook', 'description__google', 'email__facebook',
       'index__facebook', 'index__google', 'index__website',
       'language__website', 'legal_name__website', 'link__facebook',
       'page_type__facebook', 'phone__facebook', 'phone__google',
       'phone__website', 'raw_address__google', 'raw_phone__google',
       'region_clean__facebook', 'region_clean__google',
       'region_clean__website', 'root_domain__facebook', 'root_domain__google',
       'root_domain__website', 'site_name_clean__facebook',
       'site_name_clean__google', 'site_name_clean__website',
       'zip_code__facebook', 'zip_code__google'],
      dtype='object')

In [5]:
matches_full.head()

,address__facebook,address__google,category__facebook,category__google,category__website,city_clean__facebook,city_clean__google,city_clean__website,country_clean__facebook,country_clean__google,...,region_clean__google,region_clean__website,root_domain__facebook,root_domain__google,root_domain__website,site_name_clean__facebook,site_name_clean__google,site_name_clean__website,zip_code__facebook,zip_code__google
0,NaN,"3/304 Manns Rd, West Gosford NSW 2250, Australia",NaN,Promotional Merchandising,Work Clothing & Protection Equipment,NaN,Gosford,Gosford,NaN,Australia,...,New South Wales,New South Wales,NaN,wrapcitysigns.com.au,wrapcitysigns.com.au,NaN,Wrap City and Signs,Wrap City & Signs,NaN,2250
1,NaN,"Suite 228, 12, 111 Fourth Ave, St. Catharines,...",NaN,Real Estate - Agents & Managers,Real Estate - Agents & Managers,NaN,St. Catharines,St. Catharines,NaN,Canada,...,Ontario,Ontario,NaN,thereiteclub.com,thereiteclub.com,NaN,The REITE Club,The REITE Club,NaN,l2s 3p5
2,NaN,"1926 Seventh Street Louth, St. Catharines, ON ...",NaN,Plant Nurseries & Stores,Plant Nurseries & Stores,NaN,St. Catharines,St. Catharines,NaN,Canada,...,Ontario,Ontario,NaN,pioneer-pff.com,pioneer-pff.com,NaN,Pioneer Flower Farms Limited,Pioneer Flower Farms,NaN,l2r 6p9
3,NaN,"585 Carlton St, St. Catharines, ON L2M 4Y1, Ca...",NaN,Funeral Services & Cemeteries,Funeral Services & Cemeteries,NaN,St. Catharines,St. Catharines,NaN,Canada,...,Ontario,Ontario,NaN,dartefuneralhome.com,dartefuneralhome.com,NaN,George Darte Funeral Chapel,George Darte Funeral Home,NaN,l2m 4y1
4,NaN,"10 Northrup Crescent, St. Catharines, ON L2M 7...",NaN,Railroad Service Companies,Fences & Ornamental Metal Work,NaN,St. Catharines,St. Catharines,NaN,Canada,...,Ontario,Ontario,NaN,amberstairs.ca,amberstairs.ca,NaN,Amber Stairs & Railings Inc.,Amber Stairs & Railings,NaN,l2m 7m4


In [10]:
l = [np.nan, 0, 0]
print(pd.Series(l).dropna().unique()[0])

0.0


In [13]:
def match_exact(row, column_name):
    if row[column_name+'__facebook'] == row[column_name+'__google'] or \
    row[column_name+'__facebook'] == row[column_name+'__website'] or \
    row[column_name+'__google'] == row[column_name+'__website']:
        combination = [row[column_name+'__facebook']
                       , row[column_name+'__google']
                       , row[column_name+'__website']]
        return pd.Series(combination).dropna().unique()[0]
    elif pd.isna(row[column_name+'__facebook']) and \
        pd.isna(row[column_name+'__google']) and \
        pd.isna(row[column_name+'__website']):
            return np.nan
        
matches_full ['city_final'] = (matches_full[['city_clean__facebook'
                                            , 'city_clean__google'
                                            , 'city_clean__website']]
                            .apply(lambda row: match_exact(row, 'city_clean'), axis = 1)
                                  )